In [213]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

**Параметры**

In [214]:
L = [8,16,32,64,128,256]
dp = 0.001
threshold = np.array([1 - dp*i for i in range(1, 1000)])
threshold
threshold = np.flip(threshold)

**Анимация матриц**

In [215]:
def draw_matrix(images, a, the_path=[],zoom=20,borders=6):
    im = Image.new('RGB', (zoom * len(a[0]), zoom * len(a)), (153, 76, 0))
    draw = ImageDraw.Draw(im)
    for i in range(len(a)):
        for j in range(len(a[i])):
            color = (153, 76, 0)
            r = 0
            if a[i][j] == 1 or a[i][j] < 0:
                color = (255, 255, 255)

            draw.rectangle((j*zoom+r, i*zoom+r, j*zoom+zoom -
                            r-1, i*zoom+zoom-r-1), fill=color)
            
            if a[i][j] < 0:
                r = borders
                draw.rectangle((j * zoom, i * zoom , j * zoom + zoom , i * zoom + zoom ),
                            fill=(0, 0, 255))
    
    images.append(im)

**Алгоритм обнаружения связного кластера**

In [216]:
def connected_cluster_test(mtrx, name = 'cluster_test', gif_flag = False):
    images = []

    if gif_flag:
        draw_matrix(images,mtrx)

    flag = 0
    mtrx[0] = np.where(mtrx[0] == 1, -1, mtrx[0])

    if gif_flag:
        draw_matrix(images,mtrx)

    for i in range(1, len(mtrx)):  # строка

        # Вариант 1 с переходом влево-вправо на все клетки

        # Идем вниз
        mtrx[i] = np.where((mtrx[i] == 1) & (mtrx[i - 1] == -1), -1, mtrx[i])
        flag = 1 if np.any(mtrx[i] < 0) else 0

        # Для каждого столбца проверяем, есть ли сверху проход
        # Если есть, то проверяем лево и право
        for j in range(len(mtrx[i])):
            if mtrx[i][j] == -1:
                
                current_idx = j

                # Пока слева не край и пока есть свободные клетки, идем влево до упора
                while mtrx[i][current_idx - 1] == 1 and current_idx - 1 != -1:
                    # Есть проход к левой ячейке, ставим -1
                    mtrx[i][current_idx - 1] = -1
                    flag = 1
                    current_idx = current_idx - 1

                    
                current_idx = j

                # Пока справа не край и пока есть свободные клетки, идем вправо до упора
                while current_idx + 1 != len(mtrx[i]) and mtrx[i][current_idx + 1] == 1:
                    # Есть проход к правой ячейке, ставим -1
                    mtrx[i][current_idx + 1] = -1
                    flag = 1
                    current_idx = current_idx + 1

        if flag == 0:
            #print("Связной кластер обрывается!")
            break

        # Вариант 2 с переходом влево-вправо на одну клетку
        
        # mtrx[i] = np.where((mtrx[i] == 1) & (mtrx[i-1] == -1), -1, mtrx[i])
        # flag = 1 if np.any(mtrx[i] < 0) else 0

        # if mtrx[i][0] == 1 and mtrx[i][1] == -1:
        #     mtrx[i][0] = -1
        #     flag = 1

        # for j in range(1, len(mtrx[i])-1):  # столбец
        #     if mtrx[i][j] == 1 and (mtrx[i][j-1] == -1 or mtrx[i][j+1] == -1):
        #         mtrx[i][j] = -1
        #         flag = 1

        # if mtrx[i][-1] == 1 and mtrx[i][-2] == -1:
        #     mtrx[i][-1] = -1
        #     flag = 1

        # if flag == 0:
        #     #print("Связной кластер обрывается!")
        #     break


        if gif_flag:
            draw_matrix(images,mtrx)
            
    if gif_flag:
        images[0].save('img/' + name + '.gif',
                save_all=True, append_images=images[1:],
                optimize=False, duration=1000, loop=0)
    return flag


**Рекурсивный алгоритм поиска "соседей"**

In [217]:
def find_neighbours(mtrx, counter_N, arg, idx):
    i, j = idx
    mtrx[i][j] = arg
    if (i - 1) >= 0 and mtrx[i - 1][j] == 1:
        mtrx[i - 1][j] = arg
        counter_N[-1] += 1
        find_neighbours(mtrx, counter_N, arg, (i - 1, j))

    if (i + 1) < len(mtrx) and mtrx[i + 1][j] == 1:
        mtrx[i + 1][j] = arg
        counter_N[-1] += 1
        find_neighbours(mtrx, counter_N, arg, (i + 1, j))

    if (j + 1) < len(mtrx[0]) and mtrx[i][j + 1] == 1:
        mtrx[i][j + 1] = arg
        counter_N[-1] += 1
        find_neighbours(mtrx, counter_N, arg, (i, j + 1))
        
    if (j - 1) >= 0 and mtrx[i][j - 1] == 1:
        mtrx[i][j - 1] = arg
        counter_N[-1] += 1
        find_neighbours(mtrx, counter_N, arg, (i, j - 1))
    return 1


**Алгоритм расчета количества кластеров**

In [218]:
def cluster_counter(mtrx):
    counter_N = []
    arg = 2
    for j in range(len(mtrx[0])):
        if mtrx[0][j] == 1:
            counter_N.append(1)
            find_neighbours(mtrx, counter_N, arg, (0,j))
            arg += 1
    counter_N = np.array(counter_N)
    #print(counter_N)
    return len(np.where(counter_N > 2)[0])

**Алгоритм определения порога протекания**

In [219]:
np.random.seed(27)
import sys
sys.setrecursionlimit(10000)

name = 'percolation_'


tmp_counter = 0
M = [100, 500, 1000]
database = []

for k in M:
    data = {str(i): [0, 0, 0] for i in L}
    for l in L:
        for p in threshold:
            tmp_counter = 0

            for m in range(k):
                table = np.random.uniform(size=(l, l))
                table = np.where(table <= p, 0, 1)
                # print("\nПараметры:\n порог перколяции", p, "Размер матрицы",
                #       l, "Номер испытания", m, ' ', sep='\t', end='')
                tmp_counter += connected_cluster_test(
                    table, name + str(p) + '_' + str(m))
                if tmp_counter > 0:
                    break
            if tmp_counter == 0:
                data[str(l)][:-1] = [p, p + dp]
                break
            
        p_tmp = data[str(l)]
        table = np.random.uniform(size=(l, l))
        table = np.where(table <= p_tmp[0], 0, 1)
        print(table, end='\n------------------------\n')
        data[str(l)][-1] = cluster_counter(table)
    database.append(data)
print(f"\n{database}")

[[0 0 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 1 1 0 1]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 1 1 0 1]
 [1 1 1 0 1 1 1 0]
 [1 0 0 0 0 1 0 0]
 [1 1 0 0 0 1 1 0]]
------------------------
[[0 1 0 1 1 0 1 1 0 0 0 1 0 0 1 0]
 [0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1]
 [1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0]
 [1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 1]
 [1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 1]
 [0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 1]
 [1 1 1 0 1 1 1 1 0 0 0 0 1 0 1 1]
 [0 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 0]
 [1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1]
 [0 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0]
 [1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1]
 [0 0 1 0 1 1 1 1 1 1 1 0 0 1 0 1]
 [0 0 1 0 1 1 0 1 0 0 1 0 1 1 1 0]]
------------------------
[[1 1 0 ... 0 1 1]
 [0 0 1 ... 0 1 1]
 [0 0 1 ... 1 1 1]
 ...
 [1 1 1 ... 0 1 1]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 1 1 0]]
------------------------
[[0 1 0 ... 0 0 0]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 0 0 1]
 ...
 [0 0 1 ... 1 0 1]
 [1 0 0 ..

**Сохранение вывода**

In [220]:
M = 100

# вариант 1 (правильный по ВО)
{'8': (0.615, 0.616), '16': (0.517, 0.518), '32': (0.473, 0.474),
 '64': (0.449, 0.450), '128': (0.424, 0.425), '256': (0.409, 0.410)}

# вариант 2
{'8': (0.612, 0.613), '16': (0.507, 0.508), '32': (0.461, 0.462), 
 '64': (0.420, 0.421), '128': (0.396, 0.397), '256': (0.374, 0.375)}


M = 1000

# вариант 1 (правильный по ВО)
{'8': (0.709, 0.710), '16': (0.593, 0.594), '32': (0.527, 0.528),
  '64': (0.472, 0.473), '128': (0.444, 0.445), '256': (0.419, 0.420)}

# вариант 2
{'8': (0.692, 0.693), '16': (0.588, 0.589), '32': (0.508, 0.509), 
 '64': (0.437, 0.438), '128': (0.410, 0.411), '256': (0.389, 0.390)}

{'8': (0.692, 0.693),
 '16': (0.588, 0.589),
 '32': (0.508, 0.509),
 '64': (0.437, 0.438),
 '128': (0.41, 0.411),
 '256': (0.389, 0.39)}

**Тест**

In [221]:
l = 8
p = 0.5
m = 0
np.random.seed(27)
table = np.random.uniform(size=(l, l))
table = np.where(table <= p, 0, 1)
print(table, end='\n------------------------\n')
# print("\nПараметры:\n порог перколяции", p, "Размер матрицы",
#       l, "Номер испытания", m, ' ', sep='\t', end='')
connected_cluster_test(table, "percolation_" + str(p) + '_' + str(m), True)

[[0 1 1 1 0 1 1 0]
 [1 1 1 1 1 1 0 0]
 [0 1 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 1 1 0 0 1 0 0]
 [1 1 0 0 1 1 1 0]
 [1 1 0 1 1 1 1 0]
 [1 0 1 0 1 1 1 0]]
------------------------


1